# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from IPython import display

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName;", engine)
X = df.message.values
Y = pd.get_dummies(df.genre).values

In [3]:
pd.get_dummies(df.genre).head()

,direct,news,social
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [4]:
df[df.genre=='direct'].sample(15)[['genre', 'message']].values

array([['direct', 'temperature of the medium of Port-au-prince. '],
       ['direct',
        'this is not a plan to help the schools which are victims? '],
       ['direct', 'I NEED SOME INFORMATION ABOUT TEMPARATURE. '],
       ['direct', '10 1 liter bottles of water in the east village'],
       ['direct',
        'We are in the area of Petit Goave, we would like .. we need tents and medication for flu/colds.. '],
       ['direct',
        'There are 5-7 people in the government that are taking portions of the aid coming to the people '],
       ['direct',
        'I miss school, I am away from my favorite friends, I am bored, I want to go to school, I do not want to ruin my future, I want to be someone respectable in '],
       ['direct',
        "We are are in Gonaive. There are dead below and starving because all the students died in the earthquake. We are asking ( with/and? ) authorities ( about? ) our 'lean' situation here. "],
       ['direct',
        'Are the earthquakes ove

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # nromalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=2))
])

### 4. Train pipeline
* Split data into train and test sets
* Train pipeline

In [7]:
# split data
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=42)

In [8]:
# train data
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=2))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# basic model
pred_y = pipeline.predict(test_X)

In [10]:
print(classification_report(test_y, pred_y, target_names=["direct", "news", "social"]))

             precision    recall  f1-score   support

     direct       0.91      0.93      0.92      3226
       news       0.97      0.90      0.93      3912
     social       0.97      0.82      0.89       727

avg / total       0.95      0.90      0.92      7865



In [11]:
display.display_html(
    "<p style='font: italic 25px Fira Sans, serif;'> \
        The naive model report: \
        <ol style='font: italic 25px Fira Sans, serif;'> \
            <li>Average F1 Score is {0:.3f}</li> \
            <li>Average Precision Score is {1:.3f}</li> \
            <li>Average Recall Score is {2:.3f}</li> \
        </ol> \
    </p>".format( 
        f1_score(test_y, pred_y, average="micro"),
        precision_score(test_y, pred_y, average="micro"),
        recall_score(test_y, pred_y, average="micro")
    ), raw=True
)

The naive model report: Average F1 Score is 0.923 Average Precision Score is 0.944 Average Recall Score is 0.903

In [12]:
pipeline.get_params()

{'memory': None, 'steps': [('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
               oob_score=False, random_state=42, verbose=0, warm_start=False),
              n_jobs=2))], 'text_pipel

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
forest = RandomForestClassifier(random_state=42)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest))
])

In [13]:
# params
params = {
#     'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'text_pipeline__vect__max_df': (0.75, 1.0),
    'text_pipeline__vect__max_features': (5000, 100000),
#     'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [4, 6, 10],

}

In [15]:
# train model
cv = GridSearchCV(pipeline, param_grid=params)

cv.fit(train_X, train_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'text_pipeline__vect__max_features': (5000, 100000), 'clf__estimator__n_estimators': [10, 20, 30], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [4, 6, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# predict values
pred_y = cv.predict(test_X)

In [17]:
print(classification_report(test_y, pred_y, target_names=["direct", "news", "social"]))

             precision    recall  f1-score   support

     direct       0.97      0.45      0.62      3226
       news       0.83      0.96      0.89      3912
     social       1.00      0.18      0.30       727

avg / total       0.91      0.68      0.72      7865



In [18]:
display.display_html(
    "<p style='font: italic 25px Fira Sans, serif;'> \
        The model report: \
        <ol style='font: italic 25px Fira Sans, serif;'> \
            <li>Average F1 Score is {0:.3f}</li> \
            <li>Average Precision Score is {1:.3f}</li> \
            <li>Average Recall Score is {2:.3f}</li> \
        </ol> \
    </p>".format( 
        f1_score(test_y, pred_y, average="micro"),
        precision_score(test_y, pred_y, average="micro"),
        recall_score(test_y, pred_y, average="micro")
    ), raw=True
)

The model report: Average F1 Score is 0.763 Average Precision Score is 0.872 Average Recall Score is 0.678

In [19]:
cv.best_params_

{'clf__estimator__criterion': 'entropy',
 'clf__estimator__max_depth': 10,
 'clf__estimator__n_estimators': 20,
 'text_pipeline__vect__max_features': 5000}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
def chunck_tokenize(text):
    # nromalize text
    text = re.sub(r"#[a-zA-Z0-9]", "tagname", text)
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "urlholder", text)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token.lower() not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

In [21]:
def check(x):
    length = len(sent_tokenize(x))
    chunck = False

    for sentence in sent_tokenize(x):
        pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

        try:
            word, tag = pos_tags[0]

            if "VB" in tag:
                chunck = True
        except:
            continue
        
        try:
            for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                if tag == "NNP":
                    chunck = True
                    break
        except:
            continue
            
        if chunck:
            break

    return (length, chunck)

In [22]:
class WordChunckExtract(BaseEstimator, TransformerMixin):
    def starting_verb(self, text): 
        length = len(sent_tokenize(text))
        chunck = False

        for sentence in sent_tokenize(text):
            pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

            try:
                word, tag = pos_tags[0]

                if "VB" in tag:
                    chunck = True
            except:
                continue

            try:
                for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                    if tag == "NNP":
                        chunck = True
                        break
            except:
                continue

            if chunck:
                break

        return (length, chunck)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        result = pd.DataFrame()
        X_tagged = pd.Series(X).apply(self.starting_verb)
        result["length"] = X_tagged.apply(lambda x: x[0] > 1)
        result["chucnk"] = X_tagged.apply(lambda x: x[1])
        return result


In [23]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text_pipeline", Pipeline([
            ("vect", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())
        ])), 
        ("exact_chunck", WordChunckExtract())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=4))
])

In [24]:
# params
params = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [100, 150],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [12, 18, 25],
    'features__transformer_weights': (
    {'text_pipeline': 1, 'starting_verb': 0.5},
#     {'text_pipeline': 0.5, 'starting_verb': 1},
#     {'text_pipeline': 0.8, 'starting_verb': 1},
)

}

# train model
cv = GridSearchCV(pipeline, param_grid=params)

cv.fit(train_X, train_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=4))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'clf__estimator__n_estimators': [100, 150], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [12, 18, 25], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5},)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
# predict values
pred_y = cv.predict(test_X)

In [26]:
print(classification_report(test_y, pred_y, target_names=["direct", "news", "social"]))

             precision    recall  f1-score   support

     direct       0.97      0.75      0.85      3226
       news       0.89      0.97      0.93      3912
     social       0.99      0.56      0.72       727

avg / total       0.93      0.84      0.88      7865



In [28]:
display.display_html(
    "<p style='font: italic 25px Fira Sans, serif;'> \
        The improved model report: \
        <ol style='font: italic 25px Fira Sans, serif;'> \
            <li>Average F1 Score is {0:.3f}</li> \
            <li>Average Precision Score is {1:.3f}</li> \
            <li>Average Recall Score is {2:.3f}</li> \
        </ol> \
    </p>".format( 
        f1_score(test_y, pred_y, average="micro"),
        precision_score(test_y, pred_y, average="micro"),
        recall_score(test_y, pred_y, average="micro")
    ), raw=True
)

The improved model report: Average F1 Score is 0.883 Average Precision Score is 0.926 Average Recall Score is 0.844

### 9. Export your model as a pickle file

In [29]:
import pickle

In [31]:
with open("model.pickle", "wb") as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.